# 11. Neural Network Backpropagation

In [1]:
from random import seed, randrange, random
from math import exp

from functions import *

### Define functions
* Simple neural network with a single hidden layer using backpropagation

In [2]:
# Initialize a network of one hidden layer
# Input: number of inputs (features), number of neurons in hidden layer, number of outputs
# Output: network, a list of layers, each a list of dictionaries (one per neuron)
def initialize_network(n_inputs, n_hidden, n_outputs):
    # Create an array (list) of layers, each an array of node dictionaries
    network = list()
    # Hidden layer = list of dictionaries, so repeated n_hidden times
    # Each dictionary has key "weights" with value = list of input (neuron) weights (+1 for bias)
    hidden_layer = [{"weights": [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    # Since feedforward, it's now n_hidden weights per neuron in (output) layer
    # n_outputs number of nodes, since want as many as there are classes
    output_layer = [{"weights": [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

# Calculate neuron activation for an input
# Input: weights (of neuron), inputs (whether row of data or output from previous layer of neurons)
# Output: activation (the activation value itself, regardless of activation threshold)
def activate_neuron(weights, inputs):
    # Initialize activation equal to bias (assumed to be in last position)
    activation = weights[-1]
    # Multiply each input with corresponding weight, except the bias (assumed last) since constant
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

# Sigmoid transfer a neuron activation
# Input: neuron activation value
# Output: sigmoid transferred value between 0 and 1
def transfer_sigmoid(activation):
    return 1.0 / (1.0 + exp(-activation))

# Feed a row of data forward through the network
# Input: network (list of layers, each a list of dictionaries - one per neuron), row of data
# Output: list of Sigmoid transferred activations of output layer
def forward_propagate(network, row):
    # Store raw row of data in a var, which is updated to layer outputs as the data is fed forward
    inputs = row
    # Iterate over lists, one per layer, each in turn a list of dictionaries (neurons)
    for layer in network:
        # Array to collect outputs for a layer, later used as inputs for the following layer
        new_inputs = []
        # Iterate over neurons in layer, each a dictionary with key 'weights'; value list of weights
        for neuron in layer:
            # Calculate the neuron activation, given its weights and the provided input
            activation = activate_neuron(neuron["weights"], inputs)
            # Store the neuron's output, having transferred the activation via the Sigmoid function
            neuron["output"] = transfer_sigmoid(activation)
            new_inputs.append(neuron["output"])
        # Update "inputs" to the output of the last layer (rather than the raw input data)
        inputs = new_inputs
        # Data is fed forward; new "inputs" now the output of previous layer rather than raw data
    # After data is fed forward through all layers, return the final layer transferred activations
    return inputs

# Calculate the derivative of an observed neuron output, assuming Sigmoid transfer of activation
# Input: Sigmoid transferred neuron activation output
# Output: Derivative of Sigmoid transferred neuron activation output
def transfer_sigmoid_derivative(output):
    return output * (1.0 - output)

# Back-propagate error and store in neurons
# Input: network object (list of layers; lists of dictionaries (nodes)), expected outputs
# Output: None. Adds or updates "delta" (error signal) for each neuron in network (passed by ref.)
def backward_propagate_error(network, expected):
    # Iterate over network in reverse order, starting at output layer and working backwards
        # A node's weights in output layer correspond to neurons in previous hidden layer
    # "delta" = error signal for a neuron; name chosen for error's change on neuron (weight delta)
    # Error signal for neurons in hidden layer is accumulated from neurons in output layer, where
    # hidden neuron number j is also index of neuron's weight in output layer neuron['weights'][j]
    # Each i in the network is index of a layer, itself a list of dictionaries (one per neuron)
    for i in reversed(range(len(network))):
        # Extract layer, i.e. list of dictionaries
        layer = network[i]
        errors = list()
        # If not in the output layer (last i), we're in a hidden layer
        if i != len(network)-1:
            # Iterate over nodes (dictionaries) in (hidden) layer i (list)
            for j in range(len(layer)):
                # Initialize neuron error to 0
                error = 0.0
                # Iterate over all neurons in upper layer (i.e. output layer for last hidden layer)
                # Error signal of hidden layer neuron is weighted errors of all neurons in next layer
                for neuron in network[i + 1]:
                    # For neuron j in i, multiply all upper neurons' error/delta with its weight to j
                        # Neuron j's error is its total blame for upper layer errors
                    # E.g. j=1 is 2nd neuron in hidden layer. For each node in output layer,
                    # multiply its 2nd weight (connecting it to j=1 in hidden) with its error.
                    error += (neuron["weights"][j] * neuron["delta"])
                # Add neuron j's error (weight of all connected neuron errors in next layer) to list
                errors.append(error)
        # Else we are in the output layer, so error = simple difference of expected vs actual output
        # We arrive here in the first iteration, i.e. we start by calculating deltas of output layer
        else:
            # Iterate over neurons in output layer
            for j in range(len(layer)):
                # Get dictionary object that makes up the neuron
                neuron = layer[j]
                # Calculate error as difference between expected and actual output, and add to list
                errors.append(expected[j] - neuron["output"])
        # Once all layer neuron errors are calculated, update all neuron dictionaries with error
        # "delta" is final error measure, i.e. (expected - actual) * transfer_derivative
        for j in range(len(layer)):
            neuron = layer[j]
            # Calculate "delta" and assign to neuron's dictionary's key's value
            neuron["delta"] = errors[j] * transfer_sigmoid_derivative(neuron["output"])

# Update the neuron weights for a network where error signals ("delta") have been back-propagated
# Input: network with back-propagated error signals, row of input data, learning rate
# Output: None. Updated neuron weights across network, since network is passed by reference.
def update_weights(network, row, l_rate):
    # Iterate over layers in network, from input layer, through hidden, to output layer
    for i in range(len(network)):
        # Get inputs as all raw row data except last column (assumed to be the output, e.g. class)
        inputs = row[:-1]
        # If not in input layer, get input as output of previous layer, rather than row of raw data
        if i != 0:
            # Update "inputs" to list of all neuron outputs from lower/previous layer
            inputs = [neuron["output"] for neuron in network[i - 1]]
        # Iterate over all neurons in layer i
        for neuron in network[i]:
            # Iterate over inputs, whether M columns or N outputs from previous layer's N neurons
            for j in range(len(inputs)):
                # Update current neuron's weight_j by adding learning rate * error * input_j
                neuron["weights"][j] += l_rate * neuron["delta"] * inputs[j]
            # Update bias "weight"; no corresponding input to multiply with
            neuron["weights"][-1] += l_rate * neuron["delta"]

# Train a network for a fixed number of epochs
# Input: initialized network, training dataset, learning rate, number of epochs, number of outputs
# Output: None. Calls update_weights #epochs times, updating the given network passed by reference.
def train_network(network, train, l_rate, n_epoch, n_outputs):
    # Repeat for given number of epochs
    for epoch in range(n_epoch):
        # Initialize epoch sum squared error
        sum_error = 0
        # Iterate over training data
        for row in train:
            # Feed in current training row to get Sigmoid transferred activations of output neurons
            outputs = forward_propagate(network, row)
            # ONE-HOT ENCODE EXPECTED OUTPUTS
                # Ideal output would be 100% probability of correct class, 0% of all others
            # Initialize list of expected outputs to 0's
            expected = [0 for i in range(n_outputs)]
            # row[-1] = true class; output neurons are ordered so update same index of expected to 1
            expected[row[-1]] = 1
            # Increment epoch sum of squared errors of output vs one-hot encoded expectation
                # Sort of Cross Entropy (but not entirely, right?)
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            # Backward-propagate expected one-hot encoded output to update neuron error signals
            backward_propagate_error(network, expected)
            # Update neuron weights based on current row (cause of error) and updated error signals
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

# Make a prediction with a trained network
# Input: trained network, row of input data
# Output: index of network output with largest probability, i.e. the predicted class number
def predict_network(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Manages application of the back-propagation algorithm, and initialization and training of network
    # Note: can only create networks of one hidden layer
# Input: train and test datasets, learning rate, number of epochs, number of neurons in hidden layer
# Output: predicted outputs for test dataset
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    # Number of inputs, assuming last row column is the output
    n_inputs = len(train[0]) - 1
    # Number of unique outputs in training set, assumed to represent number of classes/outputs
    n_outputs = len(set([row[-1] for row in train]))
    # Initialize a network with one hidden layer
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    # Train neural network, repeatedly feed-forwarding data points and back-propagating errors
    train_network(network, train, l_rate, n_epoch, n_outputs)
    # Create list to hold predictions
    predictions = list()
    # Iterate over test data
    for row in test:
        # For the network, feed a row of input data forward through network and capture prediction
        prediction = predict_network(network, row)
        predictions.append(prediction)
    return predictions

### Testing neural net on contrived dataset

In [3]:
# Test initializing a network
seed(1)
# 2 inputs (neurons), 1 neuron in hidden layer, 2 outputs (neurons)
network = initialize_network(2, 1, 2)
# Creates one hidden layer w/ one neuron w/ three weights, output of two neurons w/ two weights
print("Initialized network architecture:")
for layer in network:
    print(layer)

# Test forward propagation
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights':
[0.4494910647887381, 0.651592972722763]}]]
row = [1, 0, None]
output = forward_propagate(network, row)
print("\nOutput from forward propagation of input:")
print(output)
# Propagates input pattern [1,0] and produces an output value that is printed
# Because the output layer has two neurons, we get a list of two numbers as output
# The actual output values are just nonsense for now, and will need to be converted (e.g. SoftMax)

# Test backpropagation of error
network = [
    [
        {'output': 0.7105668883115941,
         'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}
    ],
    [
        {'output': 0.6213859615555266,
         'weights': [0.2550690257394217, 0.49543508709194095]},
        {'output': 0.6573693455986976,
         'weights': [0.4494910647887381, 0.651592972722763]}
     ]
]
expected = [0, 1]
backward_propagate_error(network, expected)
print("\nNetwork after backpropagation of error:")
for layer in network:
    print(layer)

# Test training backprop algorithm
print("\nTraining network, with final structure:")
seed(1)
dataset = [[2.7810836,2.550537003,0],
[1.465489372,2.362125076,0],
[3.396561688,4.400293529,0],
[1.38807019,1.850220317,0],
[3.06407232,3.005305973,0],
[7.627531214,2.759262235,1],
[5.332441248,2.088626775,1],
[6.922596716,1.77106367,1],
[8.675418651,-0.242068655,1],
[7.673756466,3.508563011,1]]
# Get number of inputs, assuming last column is output
n_inputs = len(dataset[0]) - 1
# Count number of distinct classes
n_outputs = len(set([row[-1] for row in dataset]))
# Network with 2 input neurons, 2 neurons in hidden layer, 2 outputs
network = initialize_network(n_inputs, 2, n_outputs)
learning_rate = 0.5
epochs = 20
train_network(network, dataset, learning_rate, epochs, n_outputs)
for layer in network:
    print(layer)

# Test making predictions with the network
dataset = [[2.7810836,2.550537003,0],
[1.465489372,2.362125076,0],
[3.396561688,4.400293529,0],
[1.38807019,1.850220317,0],
[3.06407232,3.005305973,0],
[7.627531214,2.759262235,1],
[5.332441248,2.088626775,1],
[6.922596716,1.77106367,1],
[8.675418651,-0.242068655,1],
[7.673756466,3.508563011,1]]
network = [[{'weights': [-1.482313569067226, 1.8308790073202204, 1.078381922048799]},
{'weights': [0.23244990332399884, 0.3621998343835864, 0.40289821191094327]}],
[{'weights': [2.5001872433501404, 0.7887233511355132, -1.1026649757805829]}, {'weights':
[-2.429350576245497, 0.8357651039198697, 1.0699217181280656]}]]
print("\nMaking predictions with the trained network:")
for row in dataset:
    prediction = predict_network(network, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))
# The 100% accuracy is not unexpected considering we're predicting the same rows used to train

Initialized network architecture:
[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]

Output from forward propagation of input:
[0.6629970129852887, 0.7253160725279748]

Network after backpropagation of error:
[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]

Training network, with final structure:
>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.83

### Testing neural net on Wheat Seeds dataset

In [4]:
# With k-fold cross-validation with 5 folds, 201/5 = 40.2 = 40 records per fold
seed(1)
print("\nWheat Seeds Case Study")
dataset = load_csv("data/seeds_dataset.csv")
# Convert features from string numbers to floats
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
# Convert string class to int
str_column_to_int(dataset, len(dataset[0])-1)
# Input values vary in scale and need to be normalized to the range of 0 and 1.
# Normalize inputs
minmax = dataset_minmax(dataset)
    # Note: had to fix normalize_dataset so it didn't normalize the class label output
normalize_dataset(dataset, minmax)
# Evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
scores = evaluate_algorithm(dataset, back_propagation, n_folds, accuracy_metric, l_rate, n_epoch, n_hidden)
print('Scores ANN: %s' % scores)
print('Mean Accuracy ANN: %.3f%%' % (sum(scores)/float(len(scores))))


Wheat Seeds Case Study
Loaded data file data/seeds_dataset.csv with 210 rows and 8 columns.
>epoch=0, lrate=0.300, error=130.992
>epoch=1, lrate=0.300, error=106.670
>epoch=2, lrate=0.300, error=96.449
>epoch=3, lrate=0.300, error=84.989
>epoch=4, lrate=0.300, error=75.873
>epoch=5, lrate=0.300, error=69.593
>epoch=6, lrate=0.300, error=65.094
>epoch=7, lrate=0.300, error=61.433
>epoch=8, lrate=0.300, error=58.012
>epoch=9, lrate=0.300, error=54.499
>epoch=10, lrate=0.300, error=50.801
>epoch=11, lrate=0.300, error=47.061
>epoch=12, lrate=0.300, error=43.548
>epoch=13, lrate=0.300, error=40.466
>epoch=14, lrate=0.300, error=37.879
>epoch=15, lrate=0.300, error=35.750
>epoch=16, lrate=0.300, error=34.007
>epoch=17, lrate=0.300, error=32.572
>epoch=18, lrate=0.300, error=31.378
>epoch=19, lrate=0.300, error=30.371
>epoch=20, lrate=0.300, error=29.509
>epoch=21, lrate=0.300, error=28.761
>epoch=22, lrate=0.300, error=28.101
>epoch=23, lrate=0.300, error=27.512
>epoch=24, lrate=0.300, err

>epoch=230, lrate=0.300, error=9.200
>epoch=231, lrate=0.300, error=9.188
>epoch=232, lrate=0.300, error=9.175
>epoch=233, lrate=0.300, error=9.163
>epoch=234, lrate=0.300, error=9.151
>epoch=235, lrate=0.300, error=9.139
>epoch=236, lrate=0.300, error=9.128
>epoch=237, lrate=0.300, error=9.116
>epoch=238, lrate=0.300, error=9.104
>epoch=239, lrate=0.300, error=9.093
>epoch=240, lrate=0.300, error=9.081
>epoch=241, lrate=0.300, error=9.070
>epoch=242, lrate=0.300, error=9.059
>epoch=243, lrate=0.300, error=9.047
>epoch=244, lrate=0.300, error=9.036
>epoch=245, lrate=0.300, error=9.025
>epoch=246, lrate=0.300, error=9.014
>epoch=247, lrate=0.300, error=9.004
>epoch=248, lrate=0.300, error=8.993
>epoch=249, lrate=0.300, error=8.982
>epoch=250, lrate=0.300, error=8.972
>epoch=251, lrate=0.300, error=8.961
>epoch=252, lrate=0.300, error=8.951
>epoch=253, lrate=0.300, error=8.940
>epoch=254, lrate=0.300, error=8.930
>epoch=255, lrate=0.300, error=8.920
>epoch=256, lrate=0.300, error=8.910
>

>epoch=480, lrate=0.300, error=6.749
>epoch=481, lrate=0.300, error=6.740
>epoch=482, lrate=0.300, error=6.732
>epoch=483, lrate=0.300, error=6.723
>epoch=484, lrate=0.300, error=6.715
>epoch=485, lrate=0.300, error=6.707
>epoch=486, lrate=0.300, error=6.698
>epoch=487, lrate=0.300, error=6.690
>epoch=488, lrate=0.300, error=6.682
>epoch=489, lrate=0.300, error=6.673
>epoch=490, lrate=0.300, error=6.665
>epoch=491, lrate=0.300, error=6.657
>epoch=492, lrate=0.300, error=6.649
>epoch=493, lrate=0.300, error=6.641
>epoch=494, lrate=0.300, error=6.633
>epoch=495, lrate=0.300, error=6.625
>epoch=496, lrate=0.300, error=6.617
>epoch=497, lrate=0.300, error=6.609
>epoch=498, lrate=0.300, error=6.601
>epoch=499, lrate=0.300, error=6.593
>epoch=0, lrate=0.300, error=141.996
>epoch=1, lrate=0.300, error=110.352
>epoch=2, lrate=0.300, error=103.130
>epoch=3, lrate=0.300, error=92.147
>epoch=4, lrate=0.300, error=80.975
>epoch=5, lrate=0.300, error=72.725
>epoch=6, lrate=0.300, error=66.971
>epoc

>epoch=211, lrate=0.300, error=9.809
>epoch=212, lrate=0.300, error=9.786
>epoch=213, lrate=0.300, error=9.763
>epoch=214, lrate=0.300, error=9.740
>epoch=215, lrate=0.300, error=9.717
>epoch=216, lrate=0.300, error=9.695
>epoch=217, lrate=0.300, error=9.672
>epoch=218, lrate=0.300, error=9.650
>epoch=219, lrate=0.300, error=9.627
>epoch=220, lrate=0.300, error=9.605
>epoch=221, lrate=0.300, error=9.583
>epoch=222, lrate=0.300, error=9.561
>epoch=223, lrate=0.300, error=9.539
>epoch=224, lrate=0.300, error=9.518
>epoch=225, lrate=0.300, error=9.496
>epoch=226, lrate=0.300, error=9.475
>epoch=227, lrate=0.300, error=9.453
>epoch=228, lrate=0.300, error=9.432
>epoch=229, lrate=0.300, error=9.411
>epoch=230, lrate=0.300, error=9.391
>epoch=231, lrate=0.300, error=9.370
>epoch=232, lrate=0.300, error=9.349
>epoch=233, lrate=0.300, error=9.329
>epoch=234, lrate=0.300, error=9.309
>epoch=235, lrate=0.300, error=9.289
>epoch=236, lrate=0.300, error=9.269
>epoch=237, lrate=0.300, error=9.249
>

>epoch=472, lrate=0.300, error=6.496
>epoch=473, lrate=0.300, error=6.485
>epoch=474, lrate=0.300, error=6.474
>epoch=475, lrate=0.300, error=6.462
>epoch=476, lrate=0.300, error=6.451
>epoch=477, lrate=0.300, error=6.440
>epoch=478, lrate=0.300, error=6.429
>epoch=479, lrate=0.300, error=6.418
>epoch=480, lrate=0.300, error=6.407
>epoch=481, lrate=0.300, error=6.396
>epoch=482, lrate=0.300, error=6.385
>epoch=483, lrate=0.300, error=6.374
>epoch=484, lrate=0.300, error=6.364
>epoch=485, lrate=0.300, error=6.353
>epoch=486, lrate=0.300, error=6.342
>epoch=487, lrate=0.300, error=6.332
>epoch=488, lrate=0.300, error=6.321
>epoch=489, lrate=0.300, error=6.311
>epoch=490, lrate=0.300, error=6.301
>epoch=491, lrate=0.300, error=6.290
>epoch=492, lrate=0.300, error=6.280
>epoch=493, lrate=0.300, error=6.270
>epoch=494, lrate=0.300, error=6.260
>epoch=495, lrate=0.300, error=6.250
>epoch=496, lrate=0.300, error=6.240
>epoch=497, lrate=0.300, error=6.230
>epoch=498, lrate=0.300, error=6.220
>

...

>epoch=423, lrate=0.300, error=7.093
>epoch=424, lrate=0.300, error=7.085
>epoch=425, lrate=0.300, error=7.077
>epoch=426, lrate=0.300, error=7.070
>epoch=427, lrate=0.300, error=7.062
>epoch=428, lrate=0.300, error=7.055
>epoch=429, lrate=0.300, error=7.047
>epoch=430, lrate=0.300, error=7.039
>epoch=431, lrate=0.300, error=7.032
>epoch=432, lrate=0.300, error=7.024
>epoch=433, lrate=0.300, error=7.017
>epoch=434, lrate=0.300, error=7.009
>epoch=435, lrate=0.300, error=7.001
>epoch=436, lrate=0.300, error=6.994
>epoch=437, lrate=0.300, error=6.986
>epoch=438, lrate=0.300, error=6.979
>epoch=439, lrate=0.300, error=6.971
>epoch=440, lrate=0.300, error=6.963
>epoch=441, lrate=0.300, error=6.956
>epoch=442, lrate=0.300, error=6.948
>epoch=443, lrate=0.300, error=6.941
>epoch=444, lrate=0.300, error=6.933
>epoch=445, lrate=0.300, error=6.926
>epoch=446, lrate=0.300, error=6.918
>epoch=447, lrate=0.300, error=6.911
>epoch=448, lrate=0.300, error=6.903
>epoch=449, lrate=0.300, error=6.896
>

>epoch=150, lrate=0.300, error=11.691
>epoch=151, lrate=0.300, error=11.661
>epoch=152, lrate=0.300, error=11.631
>epoch=153, lrate=0.300, error=11.601
>epoch=154, lrate=0.300, error=11.572
>epoch=155, lrate=0.300, error=11.542
>epoch=156, lrate=0.300, error=11.513
>epoch=157, lrate=0.300, error=11.484
>epoch=158, lrate=0.300, error=11.456
>epoch=159, lrate=0.300, error=11.427
>epoch=160, lrate=0.300, error=11.399
>epoch=161, lrate=0.300, error=11.371
>epoch=162, lrate=0.300, error=11.343
>epoch=163, lrate=0.300, error=11.316
>epoch=164, lrate=0.300, error=11.289
>epoch=165, lrate=0.300, error=11.261
>epoch=166, lrate=0.300, error=11.235
>epoch=167, lrate=0.300, error=11.208
>epoch=168, lrate=0.300, error=11.181
>epoch=169, lrate=0.300, error=11.155
>epoch=170, lrate=0.300, error=11.129
>epoch=171, lrate=0.300, error=11.103
>epoch=172, lrate=0.300, error=11.078
>epoch=173, lrate=0.300, error=11.052
>epoch=174, lrate=0.300, error=11.027
>epoch=175, lrate=0.300, error=11.002
>epoch=176, 

>epoch=381, lrate=0.300, error=8.006
>epoch=382, lrate=0.300, error=7.994
>epoch=383, lrate=0.300, error=7.982
>epoch=384, lrate=0.300, error=7.970
>epoch=385, lrate=0.300, error=7.957
>epoch=386, lrate=0.300, error=7.945
>epoch=387, lrate=0.300, error=7.933
>epoch=388, lrate=0.300, error=7.921
>epoch=389, lrate=0.300, error=7.909
>epoch=390, lrate=0.300, error=7.896
>epoch=391, lrate=0.300, error=7.884
>epoch=392, lrate=0.300, error=7.872
>epoch=393, lrate=0.300, error=7.860
>epoch=394, lrate=0.300, error=7.847
>epoch=395, lrate=0.300, error=7.835
>epoch=396, lrate=0.300, error=7.822
>epoch=397, lrate=0.300, error=7.810
>epoch=398, lrate=0.300, error=7.798
>epoch=399, lrate=0.300, error=7.785
>epoch=400, lrate=0.300, error=7.773
>epoch=401, lrate=0.300, error=7.760
>epoch=402, lrate=0.300, error=7.748
>epoch=403, lrate=0.300, error=7.735
>epoch=404, lrate=0.300, error=7.723
>epoch=405, lrate=0.300, error=7.710
>epoch=406, lrate=0.300, error=7.698
>epoch=407, lrate=0.300, error=7.685
>

>epoch=113, lrate=0.300, error=11.444
>epoch=114, lrate=0.300, error=11.385
>epoch=115, lrate=0.300, error=11.326
>epoch=116, lrate=0.300, error=11.268
>epoch=117, lrate=0.300, error=11.211
>epoch=118, lrate=0.300, error=11.155
>epoch=119, lrate=0.300, error=11.100
>epoch=120, lrate=0.300, error=11.046
>epoch=121, lrate=0.300, error=10.992
>epoch=122, lrate=0.300, error=10.939
>epoch=123, lrate=0.300, error=10.887
>epoch=124, lrate=0.300, error=10.836
>epoch=125, lrate=0.300, error=10.786
>epoch=126, lrate=0.300, error=10.736
>epoch=127, lrate=0.300, error=10.687
>epoch=128, lrate=0.300, error=10.639
>epoch=129, lrate=0.300, error=10.592
>epoch=130, lrate=0.300, error=10.545
>epoch=131, lrate=0.300, error=10.499
>epoch=132, lrate=0.300, error=10.453
>epoch=133, lrate=0.300, error=10.408
>epoch=134, lrate=0.300, error=10.364
>epoch=135, lrate=0.300, error=10.320
>epoch=136, lrate=0.300, error=10.277
>epoch=137, lrate=0.300, error=10.235
>epoch=138, lrate=0.300, error=10.193
>epoch=139, 

>epoch=346, lrate=0.300, error=6.023
>epoch=347, lrate=0.300, error=6.011
>epoch=348, lrate=0.300, error=5.998
>epoch=349, lrate=0.300, error=5.986
>epoch=350, lrate=0.300, error=5.974
>epoch=351, lrate=0.300, error=5.961
>epoch=352, lrate=0.300, error=5.949
>epoch=353, lrate=0.300, error=5.937
>epoch=354, lrate=0.300, error=5.924
>epoch=355, lrate=0.300, error=5.912
>epoch=356, lrate=0.300, error=5.900
>epoch=357, lrate=0.300, error=5.888
>epoch=358, lrate=0.300, error=5.875
>epoch=359, lrate=0.300, error=5.863
>epoch=360, lrate=0.300, error=5.851
>epoch=361, lrate=0.300, error=5.839
>epoch=362, lrate=0.300, error=5.827
>epoch=363, lrate=0.300, error=5.815
>epoch=364, lrate=0.300, error=5.803
>epoch=365, lrate=0.300, error=5.790
>epoch=366, lrate=0.300, error=5.778
>epoch=367, lrate=0.300, error=5.766
>epoch=368, lrate=0.300, error=5.754
>epoch=369, lrate=0.300, error=5.742
>epoch=370, lrate=0.300, error=5.730
>epoch=371, lrate=0.300, error=5.718
>epoch=372, lrate=0.300, error=5.706
>

### Comparing neural net with other methods on Wheat Seeds

In [6]:
# Comparing against other methods:
print("\nK-Nearest Neighbours:")
scores_knn = evaluate_algorithm(dataset, k_nearest_neighbours, n_folds, accuracy_metric, 3)
print('Scores KNN: %s' % scores_knn)
print('Mean Accuracy KNN: %.3f%%' % (sum(scores_knn)/float(len(scores_knn))))

print("\nNaive Bayes:")
scores_nb = evaluate_algorithm(dataset, naive_bayes, n_folds)
print('Scores Naive Bayes: %s' % scores_nb)
print('Mean Accuracy Naive Bayes: %.3f%%' % (sum(scores_nb)/float(len(scores_nb))))

print("\nClassification and Regression Trees:")
scores_cart = evaluate_algorithm(dataset, decision_tree, n_folds, accuracy_metric, 5, 3)
print('Scores CART: %s' % scores_cart)
print('Mean Accuracy Cart: %.3f%%' % (sum(scores_cart)/float(len(scores_cart))))


K-Nearest Neighbours:
Scores KNN: [97.61904761904762, 85.71428571428571, 90.47619047619048, 95.23809523809523, 97.61904761904762]
Mean Accuracy KNN: 93.333%

Naive Bayes:
Scores Naive Bayes: [83.33333333333334, 92.85714285714286, 95.23809523809523, 100.0, 80.95238095238095]
Mean Accuracy Naive Bayes: 90.476%

Classification and Regression Trees:
Scores CART: [90.47619047619048, 100.0, 95.23809523809523, 90.47619047619048, 95.23809523809523]
Mean Accuracy Cart: 94.286%
